# Heart Disease Binary Classification

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting the heart disease of an individual based on their medical attributes.

The following approach is to be taken:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modelling
6. Experimentation

## 1. Problem Definition

> Given clinical parameters about a patient, can the model correctly predict whether or not they have heart disease?

## 2. Data

> Kaggle: https://www.kaggle.com/datasets/rishidamarla/heart-disease-prediction?resource=download

> This data is sourced from the University of California Irvine's Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

## 3. Evalutation

> The project will be pursued if a 95% accuracy at predicting whether or not a patient has heart disease during the proof of concept.

## 4. Features

### Data Dictionary

